# EMOE

In [15]:
import pandas as pd

emoe = pd.read_excel('data\\fd_emoe_serie2018.xlsx', sheet_name=None, skiprows= 1)
ek = list(emoe.keys())

In [25]:
df = emoe[ek[0]]

categories = [df.iloc[0, 1]]
categories


df['nas'] = df.isna().sum(axis = 1)
cats = df[df['nas'] == 2]
categories = categories + list(cats.iloc[:, 1])
cat_index = [0] + list(cats.index)

ldfs = [df.iloc[cat_index[i]:cat_index[i + 1],:] for i in range(len(cat_index) - 1)]


,Variable,Descripción,Observaciones,nas
6,NaN,TIPO DE OBRA EJECUTADA,NaN,2
7,p1_3_1estpo,Edificación residencial,Se refiere al tipo de obra realizada en el per...,0
8,p1_3_2estpo,Edificación no residencial,NaN,1
9,p1_3_3histpo,Infraestructura,NaN,1
10,p1_3_4estpo,Trabajos especializados,NaN,1
11,2,TOTAL DE CONTRATOS Y SUBCONTRATOS,NaN,1
12,d2_1est,Total de contratos y subcontratos,Número total de contratos y subcontratos que r...,0
13,d2_1estpo,Porcentaje de variación Total de contratos y s...,NaN,1


In [ ]:
)